# 3.3 线性回归的简洁实现

In [28]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
# torch.set_default_tensor_type('torch.FloatTensor')
torch.set_default_dtype(torch.float32)

2.1.0


## 3.3.1 生成数据集

In [29]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)
features.size(), labels.size()

(torch.Size([1000, 2]), torch.Size([1000]))

## 3.3.2 读取数据

In [30]:
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

In [31]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[-0.1016, -0.6228],
        [ 0.7134,  0.8905],
        [ 0.0216, -1.1643],
        [-1.2034, -1.5416],
        [-0.8453,  0.2372],
        [ 2.1191,  0.2780],
        [-0.4698,  2.5451],
        [-0.8467, -2.2355],
        [ 0.6408, -0.7462],
        [ 0.6338,  0.8749]]) 
 tensor([ 6.0982,  2.6009,  8.2086,  7.0269,  1.7069,  7.4882, -5.3981, 10.0988,
         8.0247,  2.5017])


## 3.3.3 定义模型

In [32]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [33]:
# 写法一



net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# # 写法二
# net = nn.Sequential()
# net.add_module('linear', nn.Linear(num_inputs, 1))
# # net.add_module ......
#
# # 写法三
# from collections import OrderedDict
# net = nn.Sequential(OrderedDict([
#           ('linear', nn.Linear(num_inputs, 1))
#           # ......
#         ]))

print(net)
print(net[0])

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [34]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.3597, 0.0983]], requires_grad=True)
Parameter containing:
tensor([-0.0866], requires_grad=True)


## 3.3.4 初始化模型参数

In [35]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [36]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.0086, 0.0071]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5 定义损失函数

In [37]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [38]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [39]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [40]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [41]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000169
epoch 2, loss: 0.000142
epoch 3, loss: 0.000129


In [42]:
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 2.0006, -3.3999]])
4.2 tensor([4.2002])


In [43]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 2.0006, -3.3999]], requires_grad=True)
Parameter containing:
tensor([4.2002], requires_grad=True)
